# Velocity Test using the VR solver

This testing procedure uses the velocity solver routine. It performs a simple t-Test on the calculated velocities.

This notebook provides two ways to run:

- Load data generated in a previous run. This let's us check the accuracy. If a split should occur or not was verified visually in a previous step.
- Generating new data. A possible way of running through SigMA as a whole. However, we can not directly verify the accuracy this way.

## Imports

In [2]:
import pandas as pd
import numpy as np
import pickle
import sys
from astropy.coordinates import SkyCoord, Galactic
import astropy.units as u
from scipy.stats import mode
from scipy.stats import ttest_ind
from calculate_velocity import calculate_velocity


sys.path.append('/home/nico/VSCodeRepos/SigMA')
from SigMA.SigMA import SigMA

## Simulated Data Loading

This data was generated in a previous run. Alternatively new data can be generated below.

In [3]:
# load data, generated for visual data analysis; this also includes the true labels and the weights generated by SigMA
df = pd.read_csv('simulated_data/df.csv', index_col=0)

# load the splits; each key/value pair contains the iteration when the split happens (key) and the cluster labels (values)
splits = pickle.load(open('simulated_data/splits.pkl', 'rb'))

# contains the labels after each split of the data
split_labels = pd.read_csv('simulated_data/labels_iteration.csv', index_col=0)

## Simulated Data Generation

In [112]:
# Courtesy of Sebastian Ratzenböck
fname = 'simulated_data/Simulated_clusters_labeled_Region0_run6.csv'

def load_alena_data(fname, n_samples=50_000, delta_perc=1, add_noise=True):
    df = pd.read_csv(fname, index_col=0)
    labels_true = df['label'].values
    cols2fit = ['ra', 'dec', 'parallax', 'pmra', 'pmdec']
    df = df.rename(columns={'plx': 'parallax'})[cols2fit]
    # Scaling functions
    sf = {
        'ra': 1/0.64,
        'dec': 1/0.65,
        'parallax': 1/0.15,
        'pmra': 1/0.49,
        'pmdec': 1/0.57      
    }
    for col in cols2fit:
        df[col] *= sf[col]

    # Add noise
    if add_noise:
        # Determine ranges
        ptp_data = df[cols2fit].max() - df[cols2fit].min()
        ranges = [
            (df[col].min() - ptp_data[col] * delta_perc, df[col].max() + ptp_data[col] * delta_perc) 
            for col in cols2fit
        ]
        # Create uniform noise
        noise = np.random.uniform(*zip(*ranges), (n_samples, len(cols2fit)))
        # Add noise to data
        df_fit_noise = pd.concat([df[cols2fit], pd.DataFrame(noise, columns=cols2fit)], ignore_index=True)
        labels_true = np.r_[labels_true, np.ones(n_samples) * -1].astype(int)
    
        return df_fit_noise, labels_true, cols2fit
    else:
        return df, labels_true, cols2fit
    
def add_rvs(df, labels, keep_fraction=0.3):
    groups_to_rvs = {
        -1: (0, 50),
        0: (20, 3),
        1: (15, 3),
        2: (25, 3),
        3: (23, 4),
        4: (30, 5),
        5: (15, 4)
    }
    for gi, (mu, sigma) in groups_to_rvs.items():
        df.loc[labels==gi, 'radial_velocity'] = np.random.normal(mu, sigma, size=np.sum(labels==gi))
        df.loc[labels==gi, 'radial_velocity_error'] = np.random.uniform(1, 5, size=np.sum(labels==gi))
        
    # Remove some of the data
    cut_drop = np.random.uniform(0, 1, size=df.shape[0]) < keep_fraction
    df.loc[cut_drop, 'radial_velocity'] = np.nan
    df.loc[cut_drop, 'radial_velocity_error'] = np.nan
    # Set pm errors
    df['pmra_error'] = 0.3
    df['pmdec_error'] = 0.3

    # Add UVW
    # SkyCoord, compute 3D velocities
    c = SkyCoord(
        ra=df['ra'].values * u.deg, 
        dec=df['dec'].values * u.deg, 
        distance=1000/df['parallax'].values * u.pc,
        pm_ra_cosdec=df['pmra'].values * u.mas/u.yr,
        pm_dec=df['pmdec'].values * u.mas/u.yr,
        radial_velocity=df['radial_velocity'].values * u.km/u.s
    )
    c = c.transform_to(Galactic())
    c.representation_type = 'cartesian'
    
    df['U'] = c.U.value
    df['V'] = c.V.value
    df['W'] = c.W.value

    return df

df, labels, cols2fit = load_alena_data(fname, n_samples=50_000, delta_perc=0.5, add_noise=True)
df = add_rvs(df, labels, keep_fraction=0.3)    

## Run Sigma & Test

In [116]:
# Courtesy of Sebastian Ratzenböck
# Run SigMA
sigma_kwargs = dict(
    cluster_features=cols2fit,
    scale_factors=None,
    # These are the default values and should be kept for now
    nb_resampling=0, max_knn_density=101,
    beta=0.99, knn_initcluster_graph=40,
    transform_function=None
)
clusterer = SigMA(data=df, **sigma_kwargs).fit(alpha=0.01, knn=15, bh_correction=True)

def produce_baseline(clusterer):
    _, p_values = clusterer.run_sigma(
        alpha=-np.inf, knn=15, return_pvalues=True
    )
    p_values = np.array(p_values)
    pv_sorted = np.sort(p_values[p_values < 0.05])
    # compute mid point between consecutive p-values
    mid_points = (pv_sorted[1:] + pv_sorted[:-1]) / 2
    # Compute vhat for first step
    clusterer.fit(alpha=0.05, knn=15, bh_correction=True)
    return mid_points, clusterer.labels_

mid_points, l0 = produce_baseline(clusterer) 

Performing gradient ascend using a 15-NN density estimation.
Updated significance threshold: 1.00e-05


In [120]:
# Mostly written by Sebastian Ratzenböck
import time

for alpha_i in mid_points:
    # fit clusterer to new alpha
    clusterer.fit(alpha=alpha_i, knn=15, bh_correction=False)
    l_i = clusterer.labels_ 
    new_clusters_id = list(set(l_i) - set(l0))

    if len(new_clusters_id) == 1: 
        # one new cluster was generated
        nc_id = new_clusters_id[0]
        part_of_old_cluster = mode(l0[l_i==nc_id], keepdims=False).mode
        print(f'New cluster: {nc_id} coming from {part_of_old_cluster}')
        
        # Compute new 3D velocities
        v_hat_i = []
        for cl_i in [part_of_old_cluster, nc_id]:
            begin = time.time()
            print('Computing velocity for cluster', cl_i)
            v_hat_i.append(
                calculate_velocity(
                    data=df.loc[l_i==cl_i],
                    rho=clusterer.weights_[l_i==cl_i]
                )
            )
            print(f'Finished Computation for cluster {cl_i} in {time.time()-begin} seconds')

        test_results = ttest_ind(v_hat_i[0], v_hat_i[1]).pvalue
        print(f'Test results: {test_results}')
        same_velo = test_results[0] > 0.05 and test_results[1] > 0.05 and test_results[2] > 0.05
        print(f'Velocity equality: {same_velo}')
        print(f'Done')
        
    elif len(new_clusters_id) > 1:
        print('More than one new cluster')
    else:
        print('No new cluster')

    l0 = np.copy(l_i)
    print('-------------------\n')

No new cluster
-------------------

New cluster: 753 coming from 4137
Computing velocity for cluster 4137
Finished Computation for cluster 4137 in 1.0461673736572266 seconds
Computing velocity for cluster 753
Finished Computation for cluster 753 in 0.5833196640014648 seconds
Test results: [2.63052388e-294 8.36175740e-131 4.99376987e-260]
Velocity equality: False
Done
-------------------

New cluster: 445 coming from 753
Computing velocity for cluster 753
Finished Computation for cluster 753 in 0.6717581748962402 seconds
Computing velocity for cluster 445
Finished Computation for cluster 445 in 0.7291834354400635 seconds
Test results: [0.82063686 0.64267368 0.94262779]
Velocity equality: True
Done
-------------------

New cluster: 8447 coming from 4137
Computing velocity for cluster 4137
Finished Computation for cluster 4137 in 1.0951950550079346 seconds
Computing velocity for cluster 8447
Finished Computation for cluster 8447 in 0.7429928779602051 seconds
Test results: [4.71754991e-09 

## Testing Procedure

In [4]:
correct_splits = 0
uvw = []
for iteration, cluster_labels in splits.items():
    print(f'--- Split of clusters {cluster_labels[0], cluster_labels[1]}---')
    # print(iteration, cluster_labels)
    # we need the data of the iteration and the weights
    uvw_cluster = []
    for cluster in cluster_labels[:2]:
        cluster_points = df[split_labels[iteration] == cluster]
        weights = df[split_labels[iteration] == cluster]['weights']
        
        uvw_cluster.append(calculate_velocity(cluster_points, weights))
    uvw.append(uvw_cluster)

    test_results = ttest_ind(uvw_cluster[0], uvw_cluster[1]).pvalue
    print(f'Test results: {test_results}')
    same_velo = test_results[0] > 0.05 and test_results[1] > 0.05 and test_results[2] > 0.05
    print(f'Velocity equality: {same_velo}')
    print(f'Should split: {"True" if cluster_labels[2] else "False"}\n')
    if same_velo != cluster_labels[2]:
        correct_splits += 1

print(f'{correct_splits} out of {len(splits)} were performed correctly')

--- Split of clusters (4137, 753)---
Test results: [1.57989464e-223 1.22514589e-105 2.76051031e-208]
Velocity equality: False
Should split: True

--- Split of clusters (753, 445)---
Test results: [1.57451876e-02 7.57170035e-13 7.07218368e-03]
Velocity equality: False
Should split: True

--- Split of clusters (4137, 8447)---
Test results: [1.05130797e-13 1.28806778e-01 1.00240851e-61]
Velocity equality: False
Should split: True

--- Split of clusters (8447, 36243)---
Test results: [2.40205044e-26 6.50704881e-29 3.39979140e-23]
Velocity equality: False
Should split: True

--- Split of clusters (4137, 22941)---
Test results: [2.11290488e-69 1.66484340e-65 1.71836023e-74]
Velocity equality: False
Should split: True

--- Split of clusters (4137, 32730)---
Test results: [2.97203186e-10 1.46927112e-18 1.02251576e-22]
Velocity equality: False
Should split: True

--- Split of clusters (753, 48293)---
Test results: [1.42545837e-07 3.65545055e-09 1.20723356e-09]
Velocity equality: False
Should sp